# Day 2

Today, we will start using nf-core pipelines to find differentially abundant genes in our dataset. 
We are using data from the following paper: https://www.nature.com/articles/s41593-023-01350-3#Sec10

1. Please take some time to read through the paper and understand their approach, hypotheses and goals.

What was the objective of the study?

The objective of the study is to find out how oxycodone withdrawal affects mice and their behaviour in the context of chronic neuropathic pain from a transcriptomic point of view.

What do the conditions mean?

oxy: oxycodone injection (treatment)


sal: saline injection (control)

What do the genotypes mean?

SNI: spared nerve injury (chronic pain)


Sham: surgery with no nerve injury (control)

Imagine you are the bioinformatician in the group who conducted this study. They hand you the raw files and ask you to analyze them.

What would you do?

Which groups would you compare to each other?

Please also mention which outcome you would expect to see from each comparison.

Given the raw data the first step is to normalize it and remove any outliers that could bias our analysis. Next up we should compare gene expression among almost all pairs of groups to gather the most information:
- SNI-Oxy vs Sham-Oxy to see if mice in oxycodone withdrawal react in the same way wether they have chronic pain or not. I would expect the withdrawal to affect both groups in a similar way but more severely in the SNI group.
- SNI-Oxy vs SNI-Sal to see in an initial stage how oxycodone affects mice in chronic pain and later in withdrawal against injured mice that never receive treatment. I would expect the oxy group to initially react better but to later be hit harder when oxycodone is no longer administered while the sal group should slowly adapt and stabilise after a difficult start.
- Sham-Oxy vs Sham-Sal to simply analyse the effect of oxycodone withdrawal in healthy mice. Less important for this study as it doesn't include a chronic pain group.

Your group gave you a very suboptimal excel sheet (conditions_runs_oxy_project.xlsx) to get the information you need for each run they uploaded to the SRA.<br>
So, instead of directly diving into downloading the data and starting the analysis, you first need to sort the lazy table.<br>
Use Python and Pandas to get the table into a more sensible order.<br>
Then, perform some overview analysis and plot the results
1. How many samples do you have per condition?
2. How many samples do you have per genotype?
3. How often do you have each condition per genotype?

In [22]:
import pandas as pd
import numpy as np

sheet = pd.read_excel("conditions_runs_oxy_project.xlsx", sheet_name="Sheet1", index_col=1)
sheet = sheet.drop(columns=["Patient", "RNA-seq", "DNA-seq"])
sheet.insert(0, "Oxy", np.where(sheet["condition: Sal"] == "x", False, True))
sheet.insert(1, "SNI", np.where(sheet["Genotype: SNI"] == "x", True, False))
sheet.drop(columns=["condition: Sal", "Genotype: SNI", "Condition: Oxy", "Genotype: Sham"], inplace=True)
print(f"There are {len(sheet)} samples")
print(f"There are {sheet['Oxy'].sum()} Oxy samples and {len(sheet) - sheet['Oxy'].sum()} Sal samples")
print(f"There are {sheet['SNI'].sum()} SNI samples and {len(sheet) - sheet['SNI'].sum()} Sham samples")
sheet

There are 16 samples
There are 8 Oxy samples and 8 Sal samples
There are 8 SNI samples and 8 Sham samples


,Oxy,SNI
Run,,
SRR23195505,False,True
SRR23195506,True,False
SRR23195507,False,False
SRR23195508,True,True
SRR23195509,True,True
SRR23195510,False,True
SRR23195511,True,False
SRR23195512,False,False
SRR23195513,False,True


They were so kind to also provide you with the information of the number of bases per run, so that you can know how much space the data will take on your Cluster.<br>
Add a new column to your fancy table with this information (base_counts.csv) and sort your dataframe according to this information and the condition.

Then select the 2 smallest runs from your dataset and download them from SRA (maybe an nf-core pipeline can help here?...)

In [26]:
bases = pd.read_csv("base_counts.csv", index_col=0)
df = bases.join(sheet, how="inner")
df.sort_values("Bases", inplace=True)
df

,Bases,Oxy,SNI
Run,,,
SRR23195516,6203117700,True,True
SRR23195511,6456390900,True,False
SRR23195517,6863840400,True,True
SRR23195505,6922564500,False,True
SRR23195508,6927786900,True,True
SRR23195519,6996050100,True,False
SRR23195509,7003550100,True,True
SRR23195514,7226808600,True,False
SRR23195510,7377388500,False,True


In [30]:
smallest_runs = df.index[:2].tolist()
import csv
csv.writer(open("ids.csv", "w")).writerows([[x] for x in smallest_runs])

While your files are downloading, get back to the paper and explain how you would try to reproduce the analysis.<br>
When you are done with this shout, so we can discuss the different ideas.